In [ ]:
from sklearn.metrics import accuracy_score # Accuracy metrics 
import pickle 

In [ ]:
# Load the model from the file
with open('model_name.pkl', 'rb') as f:
    model = pickle.load(f)

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
import glob
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose

# For static images:
data = []
img_dir = "directory path for images"  # Enter Directory of all images
data_path = os.path.join(img_dir, '*g')
files = glob.glob(data_path)
BG_COLOR = (192, 192, 192)  # gray
with mp_pose.Pose(
        static_image_mode=True,
        model_complexity=0,
        enable_segmentation=True,
        min_detection_confidence=0.1) as pose:

    for idx, file in enumerate(files):
        try:
            image = cv2.imread(file)
            image_height, image_width, _ = image.shape
            results = pose.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
            print(idx, file)

            for landmark in results.pose_landmarks.landmark:
                x = int(landmark.x * image.shape[1])
                y = int(landmark.y * image.shape[0])
                annotated_image = image.copy()
                condition = np.stack(
                    (results.segmentation_mask,) * 3, axis=-1) > 0.1
                bg_image = np.zeros(image.shape, dtype=np.uint8)
                bg_image[:] = BG_COLOR
                annotated_image = np.where(
                    condition, annotated_image, bg_image)
                mp_drawing.draw_landmarks(
                    annotated_image,
                    results.pose_landmarks,
                    mp_pose.POSE_CONNECTIONS,
                    landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())

                # Extract Pose landmarks
                data = results.pose_landmarks.landmark
                pose_row = list(np.array(
                    [[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in data]).flatten())
                del pose_row[0:40]
                row = (pose_row)

                # Make Detections
                X = pd.DataFrame([row])
                body_language_class = model.predict(X)[0]
                body_language_prob = model.predict_proba(X)[0]

                # Get status box
                cv2.rectangle(annotated_image, (0, 0),
                              (250, 60), (245, 117, 16), -1)

                # Display Class
                cv2.putText(annotated_image, 'CLASS', (95, 12),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(annotated_image, body_language_class.split(' ')[
                            0], (90, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

                # Display Probability
                cv2.putText(annotated_image, 'PROB', (15, 12),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(annotated_image, str(round(body_language_prob[np.argmax(
                    body_language_prob)], 2)), (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

                cv2.imwrite(r'directory path for result' +
                            str(idx) + '.png', annotated_image)
        except:
            pass
